In [9]:
import csv
import pandas as pd
import numpy as np
import scipy.linalg as linalg
from hodgerank_tools import *

In [10]:
symptoms = ['A', 'B', 'C','D']
#covid_data = np.array([symptoms, [3, 2, 2,5], [7, 8, 9, np.nan], [2, 2, 1,3]])
covid_df = pd.DataFrame(np.array([[3, 2, 2,5], [7, 8, 9, np.nan], [2, 2, 1,3]]), columns=symptoms)

In [11]:
print(covid_data)

[['A' 'B' 'C' 'D']
 ['3' '2' '2' '5']
 ['7' '8' '9' 'nan']
 ['2' '2' '1' '3']]


In [12]:
symptoms = ['A', 'B', 'C','D']
my_data = df_to_dict_list(covid_df)
print(my_data)

[{'A': 3.0, 'B': 2.0, 'C': 2.0, 'D': 5.0}, {'A': 7.0, 'B': 8.0, 'C': 9.0}, {'A': 2.0, 'B': 2.0, 'C': 1.0, 'D': 3.0}]


In [13]:
my_nodes = get_nodes(my_data)
print(my_nodes)

['A', 'B', 'C', 'D']


In [14]:
get_f_W(my_data, my_nodes)

(array([0. , 0. , 1.5, 0. , 2. , 2.5]),
 array([[3., 0., 0., 0., 0., 0.],
        [0., 3., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0., 0.],
        [0., 0., 0., 3., 0., 0.],
        [0., 0., 0., 0., 2., 0.],
        [0., 0., 0., 0., 0., 2.]]))

In [15]:
rank(my_data)

(  node         r
 0    D  1.500000
 1    A -0.409091
 2    B -0.500000
 3    C -0.590909,
 0.38429752066115663)